# Start EVA Server

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/00-start-eva-server.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/00-start-eva-server.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

### Launch EVA server

We use this notebook for launching the EVA server.

In [1]:
## Install EVA package if needed
## Temporary fix before we cut the new release 
%pip install "git+https://github.com/georgia-tech-db/eva.git@master" --quiet

def connect_to_server():
    from eva.interfaces.relational.db import connect
    connection = connect() 
    cursor = connection.cursor()

    return cursor

Note: you may need to restart the kernel to use updated packages.
